In [3]:
from tensorflow.keras.models import load_model

In [4]:
model = load_model('../gru_model.h5')
model.summary()

OSError: Unable to open file (file signature not found)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import inspect
import sys
pd.set_option('display.max_colwidth', None)

In [6]:
import pandas as pd
import os


def import_data(dataset):
    '''Imports sepcified dataset fomr raw_data directory.

        Parameters:
                dataset (str): energy_data or weather_features

        Returns:
                DataFrame
    '''

    path = '/home/rahmah/code/marvinliebisch/forecasting-electricity-prices'
    full_path = path + '/raw_data/' + dataset + '.csv'
    print(f"Importing {dataset} data from {full_path}...")
    df_raw = pd.read_csv(full_path)

    return df_raw


def import_clean_energy_data(dropNA=True):
    '''Returns clean energy data.

        Parameters:
                dropNA (bool): Default is True (Dropping NA rows)

        Returns:
                DataFrame
    '''
    df_energy = import_data('energy_dataset')
    df_energy['time'] = pd.to_datetime(df_energy['time'], utc=True)
    df_energy['time'] = df_energy['time'].dt.tz_convert('Europe/Madrid')
    df_energy = df_energy.set_index('time')
    df_energy = df_energy.drop(columns=['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead', 'generation marine', 'generation fossil oil shale', 'generation fossil peat', 'generation geothermal', 'generation fossil coal-derived gas', 'generation wind offshore'])
    df_energy.columns = df_energy.columns.str.replace(' ', '_')

    if dropNA == True:
        df_energy = df_energy.dropna()

    return df_energy


def import_clean_weather_data():
    '''Returns clean weather data.

        Parameters:
                None

        Returns:
                DataFrame
    '''
    df_weather = import_data('weather_features')
    df_weather['dt_iso'] = pd.to_datetime(df_weather['dt_iso'], utc=True)
    df_weather = df_weather.rename(columns={'dt_iso': 'time'})
    df_weather = df_weather.set_index('time')

    return df_weather


def import_merged_data(FeatureEngineering = True, TempMin = False, TempMax = False, WeatherIcon = False):
    '''Returns merged DataFrame containing the energy and weather data.

    Parameters:
            FeatureEngineering (bool): Default is True
            (Creates several time based features
            e.g. season, month, day of week, weekend, hour)

            TempMin, TempMax, WeatherIcon (bool): Default is False
            (Drops specified weather columns entirely)

    Returns:
            DataFrame
    '''

    df_energy = import_clean_energy_data(dropNA=True)
    df_weather = import_clean_weather_data()

    if FeatureEngineering == True:
        df_energy['hour'] = df_energy.index.hour
        df_energy['month'] = df_energy.index.month

        season_dict = {1: 'Winter',
                    2: 'Winter',
                    3: 'Spring',
                    4: 'Spring',
                    5: 'Spring',
                    6: 'Summer',
                    7: 'Summer',
                    8: 'Summer',
                    9: 'Fall',
                    10: 'Fall',
                    11: 'Fall',
                    12: 'Winter'}

        df_energy['season'] = df_energy.index.month.map(lambda x: season_dict[x])
        df_energy['weekend'] = (df_energy.index.dayofweek > 4).astype(int)
        df_energy['day_of_week'] = df_energy.index.dayofweek

    if TempMin == False:
        df_weather = df_weather.drop(columns=['temp_min'])
    if TempMax == False:
        df_weather = df_weather.drop(columns=['temp_max'])
    if WeatherIcon == False:
        df_weather = df_weather.drop(columns=['weather_icon'])

    df_weather['city_name'] = df_weather['city_name'].str.strip()
    city_array = df_weather['city_name'].unique()
    grouped = df_weather.groupby(df_weather.city_name)
    df_merged = df_energy

    for city in city_array:
        df_merged = df_merged.join(grouped
                                   .get_group(city)
                                   .drop(columns=['city_name'])
                                   .add_prefix(city.lower() + '_'))

    return df_merged



In [12]:
data_energy = import_clean_energy_data()
data_energy.head()

Importing energy_dataset data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/energy_dataset.csv...


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,generation_other_renewable,generation_solar,generation_waste,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
time,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04


In [14]:
data_energy['date'] = data_energy.index.date
data_energy.head()

,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,generation_other_renewable,generation_solar,generation_waste,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual,date
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,2015-01-01
2015-01-01 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92,2015-01-01
2015-01-01 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48,2015-01-01
2015-01-01 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32,2015-01-01
2015-01-01 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04,2015-01-01


In [20]:
data_energy['date'].value_counts().sort_index()

2015-01-01    24
2015-01-02    24
2015-01-03    24
2015-01-04    24
2015-01-05    17
              ..
2018-12-27    24
2018-12-28    24
2018-12-29    24
2018-12-30    24
2018-12-31    24
Name: date, Length: 1461, dtype: int64